In [110]:
import pandas as pd

# Read excel sheets and turn them into lists
instance_dis = pd.read_excel('OR_FP_realdata.xlsx', '距離')
instance_real_school = pd.read_excel('OR_FP_realdata.xlsx', '各里學校資訊')
instance_new_cost = pd.read_excel('OR_FP_realdata.xlsx', '新學校成本')
instance_student_number = pd.read_excel('OR_FP_realdata.xlsx', '各里學生人數')

# distance between 2 villages
distances = instance_dis.iloc[1:31, 2:32]
d = distances.values

# building costs
C = instance_new_cost.iloc[0:2,1].values

# capacity of school
N = instance_real_school.iloc[0:32,2].values

# need of teachers
T = instance_real_school.iloc[0:32,3].values

# original school numbers of each village
# S_B = instance_school_num.iloc[0:30,1].values
# S_S = instance_school_num.iloc[0:30,2].values

# student number of each village
P = instance_student_number.iloc[0:30,2].values

# number of villages
L = range(30)
W_t = 200
M = 10**5
print(N)

[   0    0    0 3838 3543 1598    0 2506    0 1154    0 1370    0  219
    0    0    0 1437   88    0 1019    0    0  104   87    0    0 1480
 1680  309]


In [111]:
from gurobipy import *

eg1 = Model("eg1")

z = []
for i in L:
    z.append([])
    for j in L:
        z[i].append(eg1.addVar(lb = 0, vtype = GRB.CONTINUOUS, name="z" + str(i+1) + str(j+1)))

x = []
for i in L:
    x.append(eg1.addVar(lb = 0, vtype = GRB.INTEGER, name="x" + str(i+1)))

y = []
for i in L:
    y.append(eg1.addVar(lb = 0, vtype = GRB.CONTINUOUS, name="y" + str(i+1)))

eg1.setObjective(quicksum(quicksum(d[i][j]*z[i][j]*P[i] + y[j]*10**5 for j in L) for i in L) + C[0]*10**8 + C[1]*10**8, GRB.MINIMIZE)
eg1.addConstrs((quicksum(z[i][j] for j in L) == 1 for i in L), "i區有多少部分去j區") # correct
eg1.addConstrs(((quicksum(z[i][j]*P[i] for i in L) <= N[j]*1.1) for j in L) ,"需求被滿足")
eg1.addConstrs(((quicksum(z[i][j]*P[i] for i in L) - N[j] <= M*x[j]) for j in L) ,"超額1")
eg1.addConstrs(((quicksum(z[i][j]*P[i] for i in L) - N[j] >= M*(x[j]-1)) for j in L) ,"超額2")
eg1.addConstrs((y[j] == x[j]*(quicksum(z[i][j]*P[i] for i in L) - N[j]) for j in L) ,"超額2")
# eg1.addConstr(quicksum(P[i] for i in L) <= quicksum((x_B[i]*N_B + x_S[i]*N_S) for i in L), "滿足需求")

{0: <gurobi.QConstr Not Yet Added>,
 1: <gurobi.QConstr Not Yet Added>,
 2: <gurobi.QConstr Not Yet Added>,
 3: <gurobi.QConstr Not Yet Added>,
 4: <gurobi.QConstr Not Yet Added>,
 5: <gurobi.QConstr Not Yet Added>,
 6: <gurobi.QConstr Not Yet Added>,
 7: <gurobi.QConstr Not Yet Added>,
 8: <gurobi.QConstr Not Yet Added>,
 9: <gurobi.QConstr Not Yet Added>,
 10: <gurobi.QConstr Not Yet Added>,
 11: <gurobi.QConstr Not Yet Added>,
 12: <gurobi.QConstr Not Yet Added>,
 13: <gurobi.QConstr Not Yet Added>,
 14: <gurobi.QConstr Not Yet Added>,
 15: <gurobi.QConstr Not Yet Added>,
 16: <gurobi.QConstr Not Yet Added>,
 17: <gurobi.QConstr Not Yet Added>,
 18: <gurobi.QConstr Not Yet Added>,
 19: <gurobi.QConstr Not Yet Added>,
 20: <gurobi.QConstr Not Yet Added>,
 21: <gurobi.QConstr Not Yet Added>,
 22: <gurobi.QConstr Not Yet Added>,
 23: <gurobi.QConstr Not Yet Added>,
 24: <gurobi.QConstr Not Yet Added>,
 25: <gurobi.QConstr Not Yet Added>,
 26: <gurobi.QConstr Not Yet Added>,
 27: <gurob

In [112]:
eg1.optimize()

Gurobi Optimizer version 11.0.1 build v11.0.1rc0 (mac64[arm] - Darwin 22.3.0 22D68)

CPU model: Apple M2 Pro
Thread count: 10 physical cores, 10 logical processors, using up to 10 threads

Optimize a model with 120 rows, 960 columns and 3660 nonzeros
Model fingerprint: 0xf63f8afd
Model has 30 quadratic constraints
Variable types: 930 continuous, 30 integer (0 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+05]
  QMatrix range    [1e+02, 2e+03]
  QLMatrix range   [1e+00, 4e+03]
  Objective range  [1e+05, 2e+07]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 1e+05]
Presolve removed 60 rows and 480 columns
Presolve time: 0.00s
Presolved: 1485 rows, 990 columns, 5580 nonzeros
Presolved model has 30 SOS constraint(s)
Variable types: 960 continuous, 30 integer (30 binary)

Root relaxation: objective 2.582031e+09, 176 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntIn

In [113]:
print ("\nz* =", eg1.objVal)


z* = 2582030963.8751874


In [114]:
for i in L:
    for j in L:
        if z[i][j].x != 0:
            print(z[i][j].varName, '=', z[i][j].x)

z18 = 1.0
z24 = 1.0
z34 = 1.0
z44 = 1.0
z55 = 1.0
z66 = 1.0
z74 = 0.9807073954983923
z729 = 0.01929260450160772
z88 = 1.0
z96 = 0.726628895184136
z98 = 0.273371104815864
z1010 = 1.0
z118 = 1.0
z1212 = 1.0
z1312 = 0.03954802259887006
z1321 = 0.96045197740113
z1414 = 1.0
z1512 = 1.0
z1614 = 0.3373493975903614
z1621 = 0.6626506024096386
z178 = 1.0
z1818 = 1.0
z198 = 0.2
z1919 = 0.8
z208 = 1.0
z2121 = 1.0
z228 = 0.02878787878787881
z2210 = 0.3196969696969697
z2218 = 0.6515151515151515
z2312 = 1.0
z2412 = 0.4799999999999003
z2424 = 0.5200000000000997
z258 = 0.45794392523364486
z2512 = 0.13551401869158877
z2525 = 0.40654205607476634
z268 = 1.0
z2729 = 0.9672131147540983
z2730 = 0.03278688524590164
z2828 = 0.9046454767726161
z2829 = 0.09535452322738387
z2929 = 1.0
z3030 = 1.0


In [115]:
fuel_cost = 0
for i in L:
    for j in L:
        fuel_cost += d[i][j]*z[i][j].x*P[i]
fuel_cost = fuel_cost
print(fuel_cost)

12030963.875187626


In [116]:
exceed = 0
for i in L:
    print(y[i].varName, '=', y[i].x)
    fuel_cost += y[i].x
print(exceed)

y1 = 0.0
y2 = 0.0
y3 = 0.0
y4 = 0.0
y5 = 0.0
y6 = 0.0
y7 = 0.0
y8 = 0.0
y9 = 0.0
y10 = 0.0
y11 = 0.0
y12 = 0.0
y13 = 0.0
y14 = 0.0
y15 = 0.0
y16 = 0.0
y17 = 0.0
y18 = 0.0
y19 = 0.0
y20 = 0.0
y21 = 0.0
y22 = 0.0
y23 = 0.0
y24 = 0.0
y25 = 0.0
y26 = 0.0
y27 = 0.0
y28 = 0.0
y29 = 0.0
y30 = 0.0
0


In [117]:
building_cost = 0
for i in range(2):
    building_cost += C[i]*10**8
print(building_cost)

2570000000.0


In [118]:
total_cost = fuel_cost + building_cost
print(total_cost)

2582030963.8751874
